In [1]:
# For downloading tweets regarding #dkpol
import sys
#!{sys.executable} -m pip install searchtweets

from searchtweets import ResultStream, gen_rule_payload, load_credentials, collect_results

my_search_args = load_credentials(filename="search_tweets_creds.yaml",
                 yaml_key="search_tweets_api",
                 env_overwrite=False)

In [2]:
rule = gen_rule_payload("dkpol", results_per_call=100) # testing with a sandbox account
print(rule)

{"query": "dkpol", "maxResults": 100}


In [3]:
# creting a result stream
rs = ResultStream(rule_payload=rule,
                  max_results=100,
                  max_pages=100,
                  **my_search_args)

print(rs)
# stating that we will collect 10,000 tweets

ResultStream: 
	{
    "username": null,
    "endpoint": "https://api.twitter.com/1.1/tweets/search/30day/dev.json",
    "rule_payload": {
        "query": "dkpol",
        "maxResults": 100
    },
    "tweetify": true,
    "max_results": 100
}


In [4]:
tweets = list(rs.stream())

In [6]:
import pandas as pd
pd.DataFrame(tweets, columns = ["user","text"])
unfiltered_testsheet = pd.DataFrame(tweets)
unfiltered_testsheet.to_csv("Data/test_payload.csv")

In [7]:
unfiltered_testsheet

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,matching_rules,possibly_sensitive,display_text_range,extended_tweet,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities
0,Mon Nov 07 12:44:56 +0000 2022,1589599784339636224,1589599784339636224,RT @uffeelbaek: Så kan det ikke siges mere tyd...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,da,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mon Nov 07 12:44:54 +0000 2022,1589599776995430403,1589599776995430403,Den Korte Avis | 24-årig afghaner anholdt i sa...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,da,[{'tag': None}],False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mon Nov 07 12:44:36 +0000 2022,1589599698494816257,1589599698494816257,RT @SteffenWM: Alex-plakater er de nye tinkahu...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,da,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mon Nov 07 12:43:57 +0000 2022,1589599537777504256,1589599537777504256,"@LRisgaard TAK! Lizette, for at tale ikke bare...","<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.578021e+18,1578021146686943233,583011881.0,583011881,...,da,[{'tag': None}],NaN,"[11, 140]","{'full_text': '@LRisgaard TAK! Lizette, for at...",NaN,NaN,NaN,NaN,NaN
4,Mon Nov 07 12:43:44 +0000 2022,1589599483947778048,1589599483947778048,RT @BosseStine: “Elon Musk har slettet tweetet...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,da,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Mon Nov 07 12:10:36 +0000 2022,1589591143616245760,1589591143616245760,EL krav til en ny regerings politik er blevet ...,"<a href=""http://twitter.com/download/iphone"" r...",True,NaN,None,NaN,None,...,da,[{'tag': None}],NaN,NaN,{'full_text': 'EL krav til en ny regerings pol...,NaN,NaN,NaN,NaN,NaN
96,Mon Nov 07 12:10:21 +0000 2022,1589591079925710849,1589591079925710849,"Mette Thiesen, jeg er helt uenig med dig i alt...","<a href=""https://mobile.twitter.com"" rel=""nofo...",True,NaN,None,NaN,None,...,da,[{'tag': None}],NaN,NaN,"{'full_text': 'Mette Thiesen, jeg er helt ueni...",NaN,NaN,NaN,NaN,NaN
97,Mon Nov 07 12:09:59 +0000 2022,1589590989270044672,1589590989270044672,"RT @HjalteOlsen: På en dag, hvor flere friheds...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,da,[{'tag': None}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,Mon Nov 07 12:09:38 +0000 2022,1589590902317920260,1589590902317920260,RT @jonasholmdk: Jeg ved godt at man skal skel...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,da,[{'tag': None}],NaN,NaN,NaN,1.588907e+18,1588907183517102081,{'created_at': 'Sat Nov 05 14:52:47 +0000 2022...,"{'url': 'https://t.co/vD04aECPyA', 'expanded':...",NaN
